In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install --upgrade tensorflow


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install nltk



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, Reshape, Softmax
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets as tfds
import nltk
from nltk.translate.bleu_score import corpus_bleu
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset, info = tfds.load('bot_adversarial_dialogue', with_info=True, as_supervised=False)
print(info)


tfds.core.DatasetInfo(
    name='bot_adversarial_dialogue',
    full_name='bot_adversarial_dialogue/dialogue_datasets/1.0.0',
    description="""
    # Bot Adversarial Dialogue Dataset.
    
    Dialogue datasets labeled with offensiveness from Bot Adversarial Dialogue task.
    The dialogues were collected by asking humans to adversarially talk to bots.
    
    
    More details in the [paper](https://arxiv.org/abs/2010.07079).
    """,
    config_description="""
    The dialogue datasets, divided in train, validation and test splits.
    """,
    homepage='https://github.com/facebookresearch/ParlAI/tree/main/parlai/tasks/bot_adversarial_dialogue',
    data_dir='/root/tensorflow_datasets/bot_adversarial_dialogue/dialogue_datasets/1.0.0',
    file_format=tfrecord,
    download_size=3.06 MiB,
    dataset_size=23.38 MiB,
    features=FeaturesDict({
        'bot_persona': Sequence(Text(shape=(), dtype=string)),
        'dialogue_id': float32,
        'episode_done': bool,
        'id': T

In [7]:
print(dataset.keys())


dict_keys(['train', 'valid', 'test'])


In [8]:
train_data = dataset['train']


In [9]:
Dataset is the loaded TFDS dataset
train_data = dataset['train']
valid_data = dataset['valid']
test_data = dataset['test']

# Extract text data from the dataset
train_texts = [example['text'].decode('utf-8') for example in tfds.as_numpy(train_data)]
valid_texts = [example['text'].decode('utf-8') for example in tfds.as_numpy(valid_data)]
test_texts = [example['text'].decode('utf-8') for example in tfds.as_numpy(test_data)]

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_texts)
valid_sequences = tokenizer.texts_to_sequences(valid_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# Pad sequences to a fixed length
maxlen = 100
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post')
valid_padded = pad_sequences(valid_sequences, maxlen=maxlen, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post')

# Print the shapes of the padded sequences
print("Shape of train_padded:", train_padded.shape)
print("Shape of valid_padded:", valid_padded.shape)
print("Shape of test_padded:", test_padded.shape)


Shape of train_padded: (69274, 100)
Shape of valid_padded: (7002, 100)
Shape of test_padded: (2598, 100)


In [10]:
# Parameters
vocab_size = 5000
lstm_units = 64
latent_dim = 100
max_sequence_length = 100

In [11]:

def build_discriminator(max_sequence_length, vocab_size, lstm_units):
    model = Sequential([
        Input(shape=(max_sequence_length, vocab_size)),
        LSTM(units=lstm_units, return_sequences=True),
        LSTM(units=lstm_units),
        Dense(1, activation='sigmoid')
    ])
    return model

def build_generator(vocab_size, lstm_units, latent_dim, sequence_length):
    model = Sequential([
        Input(shape=(latent_dim,)),
        Dense(units=lstm_units * sequence_length, activation='relu'),
        Reshape((sequence_length, lstm_units)),
        LSTM(units=lstm_units, return_sequences=True),
        Dense(vocab_size, activation='softmax')
    ])
    return model

In [12]:
# GAN
def build_gan(generator, discriminator):
    discriminator.trainable = False  
    gan_input = Input(shape=(latent_dim,))
    generated_sequences = generator(gan_input)
    gan_output = discriminator(generated_sequences)
    model = Model(gan_input, gan_output)
    return model

In [13]:
generator = build_generator(vocab_size, lstm_units, latent_dim, max_sequence_length)
discriminator = build_discriminator(max_sequence_length, vocab_size, lstm_units)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


In [ ]:
# Defining function to calculate perplexity
def calculate_perplexity(probabilities):
    return np.exp(-np.mean(np.log(probabilities)))

def predictions_to_text(predictions, tokenizer):
    text_sequences = []
    for prediction in predictions:
        text_sequence = [tokenizer.index_word.get(i, '') for i in prediction]  # List of words
        text_sequences.append(text_sequence)
    return text_sequences



# Training loop
def train_gan_with_metrics(gan, generator, discriminator, dataset, epochs, batch_size, tokenizer):
    bleu_scores = []
    perplexities = []

    for epoch in range(epochs):
        for _ in range(len(dataset) // batch_size):
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            gen_imgs = generator.predict(noise)  # Generated data

            # Fetch random batch of real texts
            indices = np.random.randint(0, len(dataset), size=batch_size)
            real_indices = dataset[indices]  
            real_imgs_one_hot = tf.one_hot(real_indices, depth=vocab_size)

            # Train discriminator on real data
            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(real_imgs_one_hot, np.ones((batch_size, 1)))

            # Train discriminator on fake data
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator via the GAN
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

            # Convert predictions to text and calculate metrics
            real_texts = predictions_to_text(tf.argmax(real_imgs_one_hot, axis=-1).numpy(), tokenizer)
            generated_texts = predictions_to_text(tf.argmax(gen_imgs, axis=-1).numpy(), tokenizer)

            # Structure references properly
            references = [[text] for text in real_texts]  # Correct structure
            bleu_score = calculate_bleu_score(references, generated_texts)
            perplexity = calculate_perplexity(np.exp(gen_imgs))  # if gen_imgs is log probabilities

            bleu_scores.append(bleu_score)
            perplexities.append(perplexity)

        print(f"Epoch {epoch+1}/{epochs}, D Loss: {d_loss}, G Loss: {g_loss}, BLEU Score: {bleu_score}, Perplexity: {perplexity}")

    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(bleu_scores)
    plt.title('BLEU Score Over Time')
    plt.xlabel('Iterations')
    plt.ylabel('BLEU Score')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(perplexities)
    plt.title('Perplexity Over Time')
    plt.xlabel('Iterations')
    plt.ylabel('Perplexity')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Function to calculate BLEU score correctly
def calculate_bleu_score(references, candidates):
    return corpus_bleu(references, candidates)  

# Train the GAN model with metrics
train_gan_with_metrics(gan, generator, discriminator, train_padded, epochs=5, batch_size=32, tokenizer=tokenizer)



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step


/usr/local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s